In [0]:
# %pip install pokemontcgsdk
import os
import json
from pokemontcgsdk import Card, RestClient

# Configure your API key from environment variable
api_key = dbutils.secrets.get(scope="my_scope", key="POKEMON_API_KEY")
if not api_key:
    raise ValueError("API key not found. Set the POKEMON_API_KEY environment variable.")
RestClient.configure(api_key)

# Fetch cards from the API (adjust pagination if needed)
cards = list(Card.where(page=1, pageSize=5))

# Convert cards to a list of full dictionaries
cards_list = []
for c in cards:
    card_dict = c.__dict__.copy()
    card_dict["abilities"] = [a.__dict__ for a in c.abilities] if c.abilities else None
    card_dict["attacks"] = [a.__dict__ for a in c.attacks] if c.attacks else None
    card_dict["images"] = c.images.__dict__ if c.images else None
    card_dict["set"] = c.set.__dict__ if c.set else None
    card_dict["tcgplayer"] = (
        c.tcgplayer.__dict__ if hasattr(c, "tcgplayer") and c.tcgplayer else None
    )
    card_dict["cardmarket"] = (
        c.cardmarket.__dict__ if hasattr(c, "cardmarket") and c.cardmarket else None
    )
    card_dict["cardmarket_prices"] = (
        c.cardmarket.prices.__dict__
        if hasattr(c, "cardmarket") and c.cardmarket and hasattr(c.cardmarket, "prices")
        else None
    )
    cards_list.append(card_dict)


# Create Spark session and DataFrame
spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(cards_list, schema=schema)

# Display schema and data
# df.printSchema()
display(df.limit(5))


# Write full JSON to file for download
# output_path = "/dbfs/tmp/pokemon_cards.json"
# with open(output_path, "w") as f:
#    json.dump(cards_list, f, indent=2)
# display(json.dumps(cards_list, indent=2))
# print(f"JSON data written to: {output_path}")

# To download from Databricks, create a markdown cell with:
# [Download JSON](files/tmp/pokemon_cards.json)

In [0]:
# %pip install pokemontcgsdk
import os
import json
from pokemontcgsdk import Card, RestClient
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    ArrayType,
    IntegerType,
)

# Configure your API key from environment variable
api_key = dbutils.secrets.get(scope="my_scope", key="POKEMON_API_KEY")
if not api_key:
    raise ValueError("API key not found. Set the POKEMON_API_KEY environment variable.")
RestClient.configure(api_key)

# Fetch all cards using the SDK
cards = list(Card.where(page=1, pageSize=1))


# Helper function to recursively convert object attributes to dicts
def obj_to_dict(obj):
    if isinstance(obj, list):
        return [obj_to_dict(item) for item in obj]
    elif hasattr(obj, "__dict__"):
        return {key: obj_to_dict(value) for key, value in obj.__dict__.items()}
    else:
        return obj


# Convert cards to a list of fully serializable dicts
cards_list = [obj_to_dict(c) for c in cards]

# Define the schema explicitly
schema = StructType(
    [
        StructField("id", StringType(), True),
        StructField("name", StringType(), True),
        StructField("supertype", StringType(), True),
        StructField("subtypes", ArrayType(StringType()), True),
        StructField("level", StringType(), True),
        StructField("hp", StringType(), True),
        StructField("types", ArrayType(StringType()), True),
        StructField("evolvesFrom", StringType(), True),
        StructField("evolvesTo", ArrayType(StringType()), True),
        StructField("rules", ArrayType(StringType()), True),
        StructField(
            "ancientTrait",
            StructType(
                [
                    StructField("name", StringType(), True),
                    StructField("text", StringType(), True),
                ]
            ),
            True,
        ),
        StructField(
            "abilities",
            ArrayType(
                StructType(
                    [
                        StructField("name", StringType(), True),
                        StructField("text", StringType(), True),
                        StructField("type", StringType(), True),
                    ]
                )
            ),
            True,
        ),
        StructField(
            "attacks",
            ArrayType(
                StructType(
                    [
                        StructField("name", StringType(), True),
                        StructField("cost", ArrayType(StringType()), True),
                        StructField("convertedEnergyCost", IntegerType(), True),
                        StructField("damage", StringType(), True),
                        StructField("text", StringType(), True),
                    ]
                )
            ),
            True,
        ),
        StructField(
            "weaknesses",
            ArrayType(
                StructType(
                    [
                        StructField("type", StringType(), True),
                        StructField("value", StringType(), True),
                    ]
                )
            ),
            True,
        ),
        StructField(
            "resistances",
            ArrayType(
                StructType(
                    [
                        StructField("type", StringType(), True),
                        StructField("value", StringType(), True),
                    ]
                )
            ),
            True,
        ),
        StructField("retreatCost", ArrayType(StringType()), True),
        StructField("convertedRetreatCost", IntegerType(), True),
        StructField(
            "set",
            StructType(
                [
                    StructField("id", StringType(), True),
                    StructField("name", StringType(), True),
                    StructField("series", StringType(), True),
                    StructField("printedTotal", IntegerType(), True),
                    StructField("total", IntegerType(), True),
                    StructField(
                        "legalities",
                        StructType(
                            [
                                StructField("unlimited", StringType(), True),
                                StructField("standard", StringType(), True),
                                StructField("expanded", StringType(), True),
                            ]
                        ),
                        True,
                    ),
                    StructField("ptcgoCode", StringType(), True),
                    StructField("releaseDate", StringType(), True),
                    StructField("updatedAt", StringType(), True),
                    StructField(
                        "images",
                        StructType(
                            [
                                StructField("symbol", StringType(), True),
                                StructField("logo", StringType(), True),
                            ]
                        ),
                        True,
                    ),
                ]
            ),
            True,
        ),
        StructField("number", StringType(), True),
        StructField("artist", StringType(), True),
        StructField("rarity", StringType(), True),
        StructField("flavorText", StringType(), True),
        StructField("nationalPokedexNumbers", ArrayType(IntegerType()), True),
        StructField(
            "legalities",
            StructType(
                [
                    StructField("unlimited", StringType(), True),
                    StructField("standard", StringType(), True),
                    StructField("expanded", StringType(), True),
                ]
            ),
            True,
        ),
        StructField("regulationMark", StringType(), True),
        StructField(
            "images",
            StructType(
                [
                    StructField("small", StringType(), True),
                    StructField("large", StringType(), True),
                ]
            ),
            True,
        ),
        StructField(
            "tcgplayer",
            StructType(
                [
                    StructField("url", StringType(), True),
                    StructField("updatedAt", StringType(), True),
                    StructField(
                        "prices",
                        StructType(
                            [
                                StructField(
                                    "normal",
                                    StructType(
                                        [
                                            StructField("low", StringType(), True),
                                            StructField("mid", StringType(), True),
                                            StructField("high", StringType(), True),
                                            StructField("market", StringType(), True),
                                            StructField(
                                                "directLow", StringType(), True
                                            ),
                                        ]
                                    ),
                                    True,
                                ),
                                StructField(
                                    "holofoil",
                                    StructType(
                                        [
                                            StructField("low", StringType(), True),
                                            StructField("mid", StringType(), True),
                                            StructField("high", StringType(), True),
                                            StructField("market", StringType(), True),
                                            StructField(
                                                "directLow", StringType(), True
                                            ),
                                        ]
                                    ),
                                    True,
                                ),
                                StructField(
                                    "1stEditionHolofoil",
                                    StructType(
                                        [
                                            StructField("low", StringType(), True),
                                            StructField("mid", StringType(), True),
                                            StructField("high", StringType(), True),
                                            StructField("market", StringType(), True),
                                            StructField(
                                                "directLow", StringType(), True
                                            ),
                                        ]
                                    ),
                                    True,
                                ),
                                StructField(
                                    "1stEditionNormal",
                                    StructType(
                                        [
                                            StructField("low", StringType(), True),
                                            StructField("mid", StringType(), True),
                                            StructField("high", StringType(), True),
                                            StructField("market", StringType(), True),
                                            StructField(
                                                "directLow", StringType(), True
                                            ),
                                        ]
                                    ),
                                    True,
                                ),
                            ]
                        ),
                        True,
                    ),
                ]
            ),
        ),
        StructField(
            "cardmarket",
            StructType(
                [
                    StructField("url", StringType(), True),
                    StructField("updatedAt", StringType(), True),
                    StructField(
                        "prices",
                        StructType(
                            [
                                StructField("averageSellPrice", StringType(), True),
                                StructField("lowPrice", StringType(), True),
                                StructField("trendPrice", StringType(), True),
                                StructField("germanProLow", StringType(), True),
                                StructField("suggestedPrice", StringType(), True),
                                StructField("reverseHoloSell", StringType(), True),
                                StructField("reverseHoloLow", StringType(), True),
                                StructField("reverseHoloTrend", StringType(), True),
                                StructField("lowPriceExPlus", StringType(), True),
                                StructField("avg1", StringType(), True),
                                StructField("avg7", StringType(), True),
                                StructField("avg30", StringType(), True),
                                StructField("reverseHoloAvg1", StringType(), True),
                                StructField("reverseHoloAvg7", StringType(), True),
                                StructField("reverseHoloAvg30", StringType(), True),
                            ]
                        ),
                        True,
                    ),
                ]
            ),
            True,
        ),
    ]
)

# Save the JSON to DBFS using Spark
#spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(cards_list, schema=schema)

# Optional: write to Delta
df.write.format("delta").mode("overwrite").saveAsTable(
    "pokemon_tcg_collection.bronze.tcg_all_cards"
)

In [0]:
import os
import json
from pokemontcgsdk import Card, RestClient
from pyspark.sql import SparkSession
from pyspark.sql.types import *

# --- Configuration ---
api_key = dbutils.secrets.get(scope="my_scope", key="POKEMON_API_KEY")
if not api_key:
    raise ValueError("API key not found. Set the POKEMON_API_KEY environment variable.")
RestClient.configure(api_key)

# --- Fetch data ---
cards = list(Card.where(page=1, pageSize=1))

# --- Helpers ---
def obj_to_dict(obj):
    if isinstance(obj, list):
        return [obj_to_dict(item) for item in obj]
    elif hasattr(obj, "__dict__"):
        return {key: obj_to_dict(value) for key, value in obj.__dict__.items()}
    else:
        return obj

cards_list = [obj_to_dict(c) for c in cards]

# --- Define schema ---
# You can infer schema with a sample if structure is not strict
# df = spark.read.json(sc.parallelize([json.dumps(cards_list[0])]))
# df.printSchema()

#schema_json = df.schema.json()
#  Your long schema here (as provided). To reduce code size, consider externalizing it if reused.
#

# --- Create Spark DataFrame ---
spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(cards_list, schema=schema)  # Consider .inferSchema if schema flexibility is okay

# --- Save to Delta Lake ---
df.display()
#df.write.format("delta").mode("overwrite").saveAsTable("pokemon_tcg_collection.bronze.tcg_all_cards")


In [0]:
import os
import json
from pokemontcgsdk import Card, RestClient
from pyspark.sql import SparkSession
from pyspark.sql.types import *

# --- Configuration ---
api_key = dbutils.secrets.get(scope="my_scope", key="POKEMON_API_KEY")
if not api_key:
    raise ValueError("API key not found. Set the POKEMON_API_KEY environment variable.")
RestClient.configure(api_key)

# Custom JSON encoder
class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj, '__dict__'):
            return obj.__dict__
        return json.JSONEncoder.default(self, obj)

# --- Fetch data ---
fee = Card.where(page=1, pageSize=1)
fee_dict = [card.__dict__ for card in fee]
foo = json.dumps(
    fee_dict,
    skipkeys=True,
    ensure_ascii=True,
    indent=4,
    sort_keys=True,
    cls=CustomEncoder
)
print(foo)

#cards = list(Card.where(page=1, pageSize=1))
#df = spark.read.json(list(Card.where(page=1, pageSize=1)))
df = spark.read.json(foo)
df.display()
    #sc.parallelize([json.dumps(cards_list[0])]))